# Extract information from GENCODE GTF

**Author**

```
Tzintzuni I. Garcia PhD
Senior Bioinformatician
Center for Translational Data Science
The University of Chicago
```
https://github.com/tzuni


This notebook extracts gene biotype information from the GTF and calculates feature coverage for all genes in the Gencode v22 annotation.

In [ ]:
import pandas as pd
import pyranges as pr

In [ ]:
# Load GTF File
gtf_file = '../gencode.v22.annotation.sorted.gtf'

colnames = ['seq', 'source', 'type', 'start', 'end', 'score', 'strand', 'phase', 'attributes']
df = pd.read_table(gtf_file, comment='#', header=None, names=colnames, )

df.head()

In [ ]:
# Just keep the gene biotype
sel_gene = df['type'] == 'gene'
gdf = df.loc[sel_gene].copy()

In [ ]:
attr_types = [
    'gene_id',
    'transcript_id',
    'gene_type',
    'gene_status',
    'gene_name',
    'transcript_type',
    'transcript_status',
    'transcript_name',
    'exon_number',
    'exon_id',
    'level',
    'tag',
    'ccdsid',
    'havana_gene',
    'havana_transcript',
    'protein_id',
    'ont',
    'transcript_support_level'
]

def parse_attrs(val):
    rec = {k: None for k in attr_types}
    parts = val.rstrip(';').split('; ')
    for p in parts:
        k, v = p.split(' ', 1)
        rec[k] = v.strip('"')
    return pd.Series(rec, index=attr_types)

In [ ]:
# This step takes some time
res = gdf['attributes'].apply(parse_attrs)

In [ ]:
# add extracted columns to gtf columns
tmp = pd.concat([gdf, res], axis=1)
tmp = tmp.drop(columns=['attributes'])
# rename columns in preparation for PyRanges
tmp = tmp.rename(columns={'seq': 'Chromosome', 'start': 'Start', 'end': 'End'})
tmp.head()

In [ ]:
# create pyranges
gpr = pr.PyRanges(tmp)

In [ ]:
# create clusters to limit overlap processing time
clusters = gpr.cluster()

In [ ]:
# Calculate coverage for each gene feature
# This takes about 9 minutes on my 4-core dell xps13

from multiprocessing import Pool

def findcluster_overlaps(cldf):
    cl_data = []
    for gene_id in cldf['gene_id']:
        sel_gene = cldf['gene_id'] == gene_id
        A = cldf.loc[sel_gene]
        B = cldf.loc[~sel_gene]
        prA = pr.PyRanges(A)
        prB = pr.PyRanges(B)
        res = prA.coverage(prB)
        cl_data.append(res.df[['gene_id', 'NumberOverlaps', 'FractionOverlaps']])
    return cl_data

with Pool(4) as p:
    cl_data = p.map(findcluster_overlaps, [grp for idx, grp in clusters.df.groupby('Cluster')], 100)

In [ ]:
# flatten nested lists and concat into a dataframe
from itertools import chain

covdf = pd.concat(chain(*cl_data))

In [ ]:
covdf.head()

In [ ]:
df2 = pd.merge(tmp, covdf, on='gene_id')
df2['overlap_other'] = df2['NumberOverlaps'] > 0
df2 = df2.rename(columns={'NumberOverlaps': 'overlap_counts', 'FractionOverlaps': 'overlap_fraction'})

In [ ]:
df2.columns

In [ ]:
df2[['gene_id', 'gene_type', 'overlap_counts', 'overlap_fraction', 'overlap_other']].to_csv('gene_types.tsv', sep='\t', header=True, index=False)